In [1]:
import pandas as pd

In [2]:
# Read CSV file 'updated_col_data.csv'
col_df = pd.read_csv('Resources/Cost_of_Living_Docs/updated_col_data.csv')
col_df.head()

,state_abv,county,family,housing_yearly,food_yearly,transportation_yearly,healthcare_yearly,misc_yearly,childcare_yearly,taxes_yearly,total_yearly,median_family_income,num_counties_in_st,st_cost_rank,st_med_aff_rank,st_income_rank
0,AL,Autauga County,1p0c,"$9,804","$4,020","$13,320","$6,120","$4,896",$0,"$7,488","$45,636","83,790",67.0,8.0,58.0,7.0
1,AL,Autauga County,1p1c,"$13,368","$5,928","$16,380","$9,072","$6,840","$6,240","$9,132","$66,960","83,790",67.0,8.0,58.0,7.0
2,AL,Autauga County,1p2c,"$13,368","$8,676","$18,840","$12,036","$7,812","$12,144","$11,388","$84,252","83,790",67.0,8.0,58.0,7.0
3,AL,Autauga County,1p3c,"$16,992","$11,592","$20,040","$14,988","$10,116","$13,956","$14,808","$102,492","83,790",67.0,8.0,58.0,7.0
4,AL,Autauga County,1p4c,"$16,992","$14,184","$20,904","$17,940","$11,040","$13,956","$17,688","$112,704","83,790",67.0,8.0,58.0,7.0


In [3]:
# Drop columns 'num_counties_in_st', 'st_cost_rank', 'st_med_aff_rank', 'st_income_rank'
mf_df = col_df.drop(columns=['family', 'num_counties_in_st', 'st_cost_rank', 'st_med_aff_rank', 'st_income_rank'])

In [4]:
# Drop rows with any NaN values
col_df.dropna(inplace=True)

In [5]:
col_df = pd.DataFrame(col_df)

In [6]:
# Remove dollar signs and commas, and convert relevant columns to numeric
columns_to_convert = ['housing_yearly', 'food_yearly', 'transportation_yearly', 
                      'healthcare_yearly', 'misc_yearly', 'childcare_yearly', 
                      'taxes_yearly', 'total_yearly', 'median_family_income']

for column in columns_to_convert:
    # Replace dollar signs and commas
    col_df[column] = col_df[column].replace({'\$': '', ',': ''}, regex=True)

    # Convert to numeric, coercing errors to NaN
    col_df[column] = pd.to_numeric(col_df[column], errors='coerce')

# Check data types of the DataFrame
print(col_df.dtypes)

# Drop rows with NaN values in specific columns
col_df.dropna(subset=columns_to_convert, inplace=True)

# Ensure that the columns are numeric after dropping NaNs
for column in columns_to_convert:
    assert pd.api.types.is_numeric_dtype(col_df[column]), f"{column} is not numeric"

# Group by state_abv and county, then calculate mean
average_df = col_df.groupby(['state_abv', 'county'])[columns_to_convert].mean().reset_index()

state_abv                 object
county                    object
family                    object
housing_yearly             int64
food_yearly                int64
transportation_yearly      int64
healthcare_yearly          int64
misc_yearly                int64
childcare_yearly           int64
taxes_yearly               int64
total_yearly               int64
median_family_income       int64
num_counties_in_st       float64
st_cost_rank             float64
st_med_aff_rank          float64
st_income_rank           float64
dtype: object


In [7]:
average_df.head()

,state_abv,county,housing_yearly,food_yearly,transportation_yearly,healthcare_yearly,misc_yearly,childcare_yearly,taxes_yearly,total_yearly,median_family_income
0,AK,Aleutians East Borough,15295.2,13826.4,18463.2,20150.4,10312.8,13478.4,13345.2,104871.6,79922.0
1,AK,Aleutians West Census Area,22203.6,13826.4,11754.0,20150.4,12756.0,15972.0,14937.6,111597.6,111607.0
2,AK,Anchorage Municipality,18061.2,13278.0,17071.2,17029.2,11096.4,18535.2,14593.2,109669.2,115272.0
3,AK,Bethel Census Area,22770.0,14637.6,11872.8,19977.6,13244.4,16077.6,15504.0,114086.4,67266.0
4,AK,Bristol Bay Borough,14671.2,13826.4,21236.4,20150.4,10092.0,13077.6,13857.6,106911.6,104688.0


In [8]:
pct_diff_df = pd.read_csv('Resources/Percent_Change_Data/county_pct_diff.csv')
pct_diff_df.head()

,Unnamed: 0,state_abv,county,pct_diff
0,1,AL,Autauga County,1.03
1,2,AL,Baldwin County,2.83
2,3,AL,Barbour County,-0.47
3,4,AL,Bibb County,-0.54
4,5,AL,Blount County,0.50


In [9]:
merge_df = pd.merge(average_df, pct_diff_df, on=['state_abv', 'county'], how='inner')

In [10]:
merge_df.head()

,state_abv,county,housing_yearly,food_yearly,transportation_yearly,healthcare_yearly,misc_yearly,childcare_yearly,taxes_yearly,total_yearly,median_family_income,Unnamed: 0,pct_diff
0,AK,Aleutians East Borough,15295.2,13826.4,18463.2,20150.4,10312.8,13478.4,13345.2,104871.6,79922.0,68,0.90
1,AK,Aleutians West Census Area,22203.6,13826.4,11754.0,20150.4,12756.0,15972.0,14937.6,111597.6,111607.0,69,1.38
2,AK,Anchorage Municipality,18061.2,13278.0,17071.2,17029.2,11096.4,18535.2,14593.2,109669.2,115272.0,70,-0.32
3,AK,Bethel Census Area,22770.0,14637.6,11872.8,19977.6,13244.4,16077.6,15504.0,114086.4,67266.0,71,-0.19
4,AK,Bristol Bay Borough,14671.2,13826.4,21236.4,20150.4,10092.0,13077.6,13857.6,106911.6,104688.0,72,-2.99


In [11]:
# Find mean for 'pct_diff' column
mean_pct_diff = pct_diff_df['pct_diff'].mean()

print(f"Mean of 'pct_diff' column: {mean_pct_diff}")

Mean of 'pct_diff' column: 0.2882824427480916


In [12]:
# Step 1: Convert pct_diff to binary values in merge_df
merge_df['pct_diff_binary'] = merge_df['pct_diff'].apply(lambda x: 0 if x > mean_pct_diff else 1)

In [13]:
merge_df.head()

,state_abv,county,housing_yearly,food_yearly,transportation_yearly,healthcare_yearly,misc_yearly,childcare_yearly,taxes_yearly,total_yearly,median_family_income,Unnamed: 0,pct_diff,pct_diff_binary
0,AK,Aleutians East Borough,15295.2,13826.4,18463.2,20150.4,10312.8,13478.4,13345.2,104871.6,79922.0,68,0.90,0
1,AK,Aleutians West Census Area,22203.6,13826.4,11754.0,20150.4,12756.0,15972.0,14937.6,111597.6,111607.0,69,1.38,0
2,AK,Anchorage Municipality,18061.2,13278.0,17071.2,17029.2,11096.4,18535.2,14593.2,109669.2,115272.0,70,-0.32,1
3,AK,Bethel Census Area,22770.0,14637.6,11872.8,19977.6,13244.4,16077.6,15504.0,114086.4,67266.0,71,-0.19,1
4,AK,Bristol Bay Borough,14671.2,13826.4,21236.4,20150.4,10092.0,13077.6,13857.6,106911.6,104688.0,72,-2.99,1


In [14]:
merge_df.to_csv('binary_county_pct_diff.csv', index=True)